## Import Packages

In [1]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn import metrics

In [2]:
random_seed = 8022022 # or any of your favorite number 
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

## Read dataset

* Make sure your input dataset included `participant_id`,`specimen`,`collect_wk`,`was_preterm`,`was_early_preterm`. Other variables should be co-variates 

In [69]:
#replace 'test/metadata_imputed.csv' with the path to your input file

mydata = pd.read_csv('data/combo_clean_data.csv', delimiter=',')
mydata = pd.DataFrame(mydata)
print(mydata.shape)
mydata.head(5)


(2461, 11)


,specimen,participant_id,collect_wk,project,was_preterm,was_early_preterm,age_imp,shannon,bwpd,CST,Lactobacillus
0,A00001-05,A00001,33,A,False,False,27,1.00000,0.00000,III,0.797900
1,A00002-01,A00002,38,A,False,False,24,1.96362,2.62894,III,0.805641
2,A00003-02,A00003,30,A,False,False,32,1.00000,0.00000,II,0.963299
3,A00004-08,A00004,27,A,False,False,25,1.00000,0.00000,III,0.927544
4,A00004-12,A00004,29,A,False,False,25,6.94884,2.78896,III,0.806593


## Data subset/type conversion if necessary

In [70]:
mydata["project"] = mydata["project"].astype('category')
mydata["CST"] = mydata["CST"].astype('category')
mydata['was_preterm'] = mydata['was_preterm'].astype('int8')
mydata['was_early_preterm'] = mydata['was_early_preterm'].astype('int8')
mydata.dtypes

specimen               object
participant_id         object
collect_wk              int64
project              category
was_preterm              int8
was_early_preterm        int8
age_imp                 int64
shannon               float64
bwpd                  float64
CST                  category
Lactobacillus         float64
dtype: object

## Subsetting dataset for different outcome

In [71]:
# only keep the rows with collect_wk < 32 for Preterm task
mydata_preterm = mydata.loc[mydata['collect_wk']<=32,]
print(mydata_preterm.shape)
# only keep the rows with collect_wk < 28 for Early preterm task
mydata_epreterm = mydata.loc[mydata['collect_wk']<=28,]
print(mydata_epreterm.shape)
print(mydata_preterm['CST'].dtypes)

(2077, 11)
(1765, 11)
category


## Define functions for pytorch input

### feature transforming function

In [84]:
def feature_transform(data, var_name, id_list, out_var = "was_preterm"):
    
    temp_data = data.copy()
    
    ##check argument validity
    if out_var != "was_preterm" and out_var != "was_early_preterm":
        raise ValueError("out_var must be was_preterm or was_early_preterm")
    if var_name not in list(data.columns):
        raise ValueError("var_name must be in column names of data")
        
    ##get data type
    var_type = data[var_name].dtypes
    
    if var_type == "category":
        temp_data[var_name] = pd.factorize(data[var_name])[0] + 1
    
    ##get pivot table of features
    temp_data_wide = temp_data.pivot_table(index = ['participant_id'], columns = 'collect_wk', values = var_name).sort_index(axis = 0)
    temp_data_wide = temp_data_wide.sort_index(axis=1)
    
    ##if categorical impute with mode of each outcome group, continuous with mean
    if var_type == "category":
        temp_data_wide = temp_data_wide.apply(lambda x: x.fillna(x.mode().iloc[0,]), axis = 1)
    else:
        temp_data_wide = temp_data_wide.apply(lambda x: x.fillna(x.mean()), axis = 1)
    
    return temp_data_wide.loc[id_list]
    

### outcome transforming function

In [7]:
def outcome_transform(data, id_list, multi_outcome = True, out_var = "was_preterm",label_smooth = True):
    outcome_data = data.groupby('participant_id').first().sort_index(axis = 0)[out_var]
    max_week_per = data.groupby('participant_id')['collect_wk'].max().sort_index()
    max_week = max_week_per.max()
    id_all = list(max_week_per.index)
    
    
    if multi_outcome:
        if label_smooth:
            label_list = [np.concatenate((np.linspace(0.5,0,max_week_per[id]),np.repeat(0,max_week-max_week_per[id]))) \
                          if outcome_data[id] == 0 else \
                         np.concatenate((np.linspace(0.5,1,max_week_per[id]),np.repeat(1,max_week-max_week_per[id]))) \
                         for id in id_all]
            temp_y = pd.DataFrame(label_list,columns = np.arange(1,max_week+1,1), index = max_week_per.index)
            return temp_y.loc[id_list]
        else:
            temp_y = data.pivot_table(index=['participant_id'], columns='collect_wk', values= out_var).sort_index(axis = 0)
            # sort by collect_wk
            temp_y = temp_y.sort_index(axis=1)
            temp_y = temp_y.apply(lambda row: row.fillna(row.mean()), axis=1)
            return temp_y.loc[id_list]
    else:
        return data.groupby('participant_id').first().sort_index(axis = 0)[out_var][id_list]

### ternsor generator

In [8]:
def tensor_generator(data, id_list, features, out_var = "was_preterm", label_smooth = True, multi_outcome = True):
    X_matrix = [feature_transform(data, var_name,id_list, out_var).to_numpy() for var_name in features]
    y_matrix = outcome_transform(data, id_list,multi_outcome,out_var,label_smooth).to_numpy()
    
    input_X = torch.from_numpy(np.dstack(X_matrix).astype('float32'))
    
    if multi_outcome:
        input_y = torch.from_numpy(np.dstack((y_matrix,1-y_matrix)).astype('float32'))
    else:
        input_y = torch.from_numpy(np.vstack((y_matrix,1-y_matrix)).T.astype('float32'))
    
    return input_X, input_y


## Define models

In [9]:
# Define RNN model
class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers = 1,drop_prob=0.2):
        
        """
            parameters:
                input_dim: dimensions of input data (# features)
                hidden_dim: dimensions of hidden layer
                output_dim: dimensions of output layer (should be two in our analysis)
                n_layers: number of layers for GRU structure, default is 1, 2 means stacked GRU
                drop_prob: dropout probability

        """
        
        #inherit from super class
        super(RNNModel, self).__init__()
        
        #define parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        #define layers
        
        ##GRU layers
        self.rnn = nn.RNN(input_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        ##fully connected layer(use one linear layer first, later can customize)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state
        hidden = self.init_hidden(batch_size)
        
        out, hidden = self.rnn(x, hidden)
        
        #pass out to fully connected layer
        out = self.fc(out.reshape(-1,out.shape[-1]))
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        return torch.zeros((self.n_layers,batch_size, self.hidden_dim), device = device)
    


In [10]:
# Define GRU model
class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers = 1,drop_prob=0.2):
        
        """
            parameters:
                input_dim: dimensions of input data (# features)
                hidden_dim: dimensions of hidden layer
                output_dim: dimensions of output layer (should be two in our analysis)
                n_layers: number of layers for GRU structure, default is 1, 2 means stacked GRU
                drop_prob: dropout probability

        """
        
        #inherit from super class
        super(GRUModel, self).__init__()
        
        #define parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        #define layers
        
        ##GRU layers
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        ##fully connected layer(use one linear layer first, later can customize)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state
        hidden = self.init_hidden(batch_size)
        
        out, hidden = self.gru(x, hidden)
        
        #pass out to fully connected layer
        out = self.fc(out.reshape(-1,out.shape[-1]))
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        return torch.zeros((self.n_layers,batch_size, self.hidden_dim), device = device)
    


In [11]:
# Define LSTM model

class LSTMModel(nn.Module):
    
    """
        parameters:
            input_dim: dimensions of input data (# features)
            hidden_dim: dimensions of hidden layer
            output_dim: dimensions of output layer (should be two in our analysis)
            n_layers: number of layers for GRU structure, default is 1, 2 means stacked GRU
            drop_prob: dropout probability

    """
        
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers = 1, drop_prob=0.2):
        
        #inherit from super class
        super(LSTMModel, self).__init__()
        
        #define parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        #define layers
        
        ##LSTM layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        
        ##fully connected layer(use one linear layer first, later can customize)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state
        hidden = self.init_hidden(batch_size)
        
        out, hidden = self.lstm(x, hidden)
        
        #pass out to fully connected layer
        out = self.fc(out.reshape(-1,out.shape[-1]))
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        return (torch.zeros((self.n_layers,batch_size, self.hidden_dim),device = device),
                torch.zeros((self.n_layers,batch_size, self.hidden_dim),device = device))

In [12]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [13]:
# define train function

def dataloader(X, y, batch_size = 100):
    data = TensorDataset(X, y)
    
    if batch_size > X.shape[0]:
        batch_size = X.shape[0]
    
    loader = DataLoader(data, shuffle = True, batch_size = batch_size, drop_last = True)
    return loader


def train_epoch(train_loader, validation_loader, learn_rate, \
                hidden_dim, n_layers, drop_prob, \
                device = device, EPOCHS = 100, output_dim = 2, method = "RNN"):
    
    #input_dim
    
    input_dim = next(iter(train_loader))[0].shape[2]
    
    #instantiating the models
    if method == "RNN":
        model = RNNModel(input_dim, hidden_dim, output_dim,n_layers = n_layers,drop_prob=drop_prob)
    if method == "GRU":
        model = GRUModel(input_dim, hidden_dim, output_dim,n_layers = n_layers,drop_prob=drop_prob)
    elif method == "LSTM":
        model = LSTMModel(input_dim, hidden_dim, output_dim,n_layers = n_layers,drop_prob=drop_prob)
        
    model.to(device)
    
    # loss criterion and optimizer

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr = learn_rate)
    
    train_loss = []
    val_loss = []
    # train model
    
    model.train()
    
    print('Starting training of {} model'.format(method))
    
    #Start training loop
    
    for epoch in range(1, EPOCHS + 1):
        
        batch_train_losses = []
        batch_val_losses = []
        
        for x, label in train_loader:
            
            x, label = x.to(device), label.to(device)
            
            model.zero_grad()
            
            label = label.reshape(-1,label.shape[-1])
        
            predictions = model(x)[0]
        
            predictions = predictions.to(device)
        
            loss = criterion(predictions, label)
            
            batch_train_losses.append(loss.detach().numpy())
            # backpropagation
            loss.backward() 
            # Updates the weights accordingly
            optimizer.step()
        
        train_loss.append(np.mean(batch_train_losses))
        
        with torch.no_grad():
            
            model.eval()
            
            for x, label in validation_loader:
                
                x, label = x.to(device), label.to(device)
                
                label = label.reshape(-1,label.shape[-1])
                
                out = model(x)[0]
                
                out = out.to(device)
                
                loss_val = criterion(out, label)
                
                batch_val_losses.append(loss_val.numpy())
        
        val_loss.append(np.mean(batch_val_losses))
        
        if epoch%10 == 0:
            print('Epoch: {}/{}.............'.format(epoch, EPOCHS), end=' ')
            print("Train Loss: {:.4f}".format(train_loss[epoch-1]))
            print("Validation Loss: {:.4f}".format(val_loss[epoch-1]))
            
        ##stopping rule?
        
    return model,train_loss,val_loss


In [14]:
def test_metrics(model, data, id_list, features, out_var = "was_preterm", multi_outcome = True):
    
    test_X = tensor_generator(data, id_list,features, out_var, label_smooth = False, multi_outcome = multi_outcome)[0]
    
    test_y = outcome_transform(data ,id_list, multi_outcome = False, out_var = "was_preterm",label_smooth = False).to_numpy()

    model.eval()
    
    if multi_outcome:
        out = model(test_X)[0]
        predicted_props = nn.functional.softmax(out.reshape(test_X.shape[0],test_X.shape[1],2),dim = 2)[:,-1,0].detach().numpy()
        predicted_labels = 1*(predicted_props >0.5)
    else:
        out = model(test_X)
        predicted_props = nn.functional.softmax(out, dim = 1)[:,0].detach().numpy()
        predicted_labels = 1*(predicted_props >0.5)
    
    
    result_tab = pd.DataFrame(data = [predicted_props,predicted_labels,test_y],
                             columns = id_list,
                             index= ['predicted_prop','predicted_y','y']).T
    
    acc = metrics.accuracy_score(test_y, predicted_labels, normalize=False) / float(test_y.size)
    confusion = metrics.confusion_matrix(test_y, predicted_labels)
    

    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    
    specificity = TN / (TN + FP)
    sensitivity = TP / (TP + FN)
    precision = TP/(TP + FP)
    
    auc = metrics.roc_auc_score(test_y, predicted_props)
    
    
    return acc,sensitivity,specificity,auc,precision,result_tab


In [82]:
def project_cross_val(features,train_ratio = 0.7, learn_rate = 0.01,hidden_dim = 18, n_layers = 1, device = device, \
                      batch_size = 1000,drop_prob = 0.2, EPOCHS = 1000, output_dim = 2, method = "RNN", \
                      out_var = "was_preterm",label_smooth= True, multi_outcome= True):
    
    ##check argument validity
    if out_var != "was_preterm" and out_var != "was_early_preterm":
        raise ValueError("outcome must be was_preterm or was_early_preterm")
    
    ##obtain data by goal
    if out_var == "was_preterm":
        data = mydata_preterm
    else:
        data = mydata_epreterm
        
    project_list = data['project'].unique()
     ##accuracy, sen, spe, auc score
    train_loss = {}
    val_loss = {}
    acc = {}
    sen = {}
    spe = {}
    auc = {}
    precision = {}
    result_tabs = {}
    
    
    for project in project_list:
        train_list = data[data['project']!= project].groupby('participant_id').first().sort_index(axis = 0)
        train_id = list(train_list.groupby(out_var).sample(frac = train_ratio, random_state = 100).sort_index(axis = 0).index)
        val_id = np.setdiff1d(list(train_list.index),train_id)
        test_id = list(data[data['project']== project].groupby('participant_id').first().sort_index(axis = 0).index)
        train_x, train_y = tensor_generator(data,train_id,features = features, \
                                            out_var= "was_preterm", \
                                            label_smooth= label_smooth, multi_outcome= multi_outcome)
        val_x, val_y = tensor_generator(data,val_id,features = features, \
                                            out_var= "was_preterm", \
                                            label_smooth= label_smooth, multi_outcome= multi_outcome)
        
        train_dataloader = dataloader(train_x, train_y,batch_size =  batch_size)
        val_dataloader = dataloader(val_x, val_y,batch_size =  batch_size)
        model, train_loss[project],val_loss[project] = train_epoch(train_dataloader,val_dataloader, learn_rate = learn_rate, \
                hidden_dim = hidden_dim, n_layers = n_layers, drop_prob = drop_prob, \
                device = device, EPOCHS = EPOCHS, output_dim = output_dim, method = method)
        
        acc[project],sen[project],spe[project],auc[project],precision[project],result_tabs[project] = \
        test_metrics(model, data,test_id,features, out_var = out_var, multi_outcome = multi_outcome)
    
    return train_loss,val_loss, result_tabs ,pd.DataFrame([acc,sen,spe,precision,auc], \
                                                 index = ['accuracy','sensitivity','specificity','precision','AUC'])
    
   
    

In [87]:
train_loss_GRU,val_loss_GRU,result_tabs_GRU,metrics_GRU = project_cross_val(['shannon','bwpd','CST','Lactobacillus','age_imp'], \
                                                    train_ratio = 0.8,learn_rate = 0.01,hidden_dim = 10, n_layers = 2, device = device, \
                                                    batch_size = 1000, drop_prob = 0,EPOCHS = 130, output_dim = 2, method = "GRU", \
                                                    out_var = "was_preterm",label_smooth= True, multi_outcome= True)

Starting training of GRU model
Epoch: 10/130............. Train Loss: 0.1706
Validation Loss: 0.1629
Epoch: 20/130............. Train Loss: 0.1346
Validation Loss: 0.1396
Epoch: 30/130............. Train Loss: 0.1234
Validation Loss: 0.1321
Epoch: 40/130............. Train Loss: 0.1217
Validation Loss: 0.1296
Epoch: 50/130............. Train Loss: 0.1205
Validation Loss: 0.1310
Epoch: 60/130............. Train Loss: 0.1194
Validation Loss: 0.1318
Epoch: 70/130............. Train Loss: 0.1183
Validation Loss: 0.1324
Epoch: 80/130............. Train Loss: 0.1173
Validation Loss: 0.1340
Epoch: 90/130............. Train Loss: 0.1162
Validation Loss: 0.1377
Epoch: 100/130............. Train Loss: 0.1153
Validation Loss: 0.1372
Epoch: 110/130............. Train Loss: 0.1145
Validation Loss: 0.1377
Epoch: 120/130............. Train Loss: 0.1137
Validation Loss: 0.1381
Epoch: 130/130............. Train Loss: 0.1129
Validation Loss: 0.1385
Starting training of GRU model
Epoch: 10/130...........

C:\Users\sxiao15\AppData\Local\Temp\ipykernel_10904\4224545255.py:34: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Starting training of GRU model
Epoch: 10/130............. Train Loss: 0.1451
Validation Loss: 0.1477
Epoch: 20/130............. Train Loss: 0.1290
Validation Loss: 0.1275
Epoch: 30/130............. Train Loss: 0.1276
Validation Loss: 0.1255
Epoch: 40/130............. Train Loss: 0.1262
Validation Loss: 0.1238
Epoch: 50/130............. Train Loss: 0.1249
Validation Loss: 0.1221
Epoch: 60/130............. Train Loss: 0.1239
Validation Loss: 0.1213
Epoch: 70/130............. Train Loss: 0.1235
Validation Loss: 0.1206
Epoch: 80/130............. Train Loss: 0.1232
Validation Loss: 0.1197
Epoch: 90/130............. Train Loss: 0.1229
Validation Loss: 0.1195
Epoch: 100/130............. Train Loss: 0.1222
Validation Loss: 0.1193
Epoch: 110/130............. Train Loss: 0.1204
Validation Loss: 0.1189
Epoch: 120/130............. Train Loss: 0.1191
Validation Loss: 0.1199
Epoch: 130/130............. Train Loss: 0.1180
Validation Loss: 0.1194


C:\Users\sxiao15\AppData\Local\Temp\ipykernel_10904\4224545255.py:34: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


Starting training of GRU model
Epoch: 10/130............. Train Loss: 0.1286
Validation Loss: 0.1245
Epoch: 20/130............. Train Loss: 0.1254
Validation Loss: 0.1199
Epoch: 30/130............. Train Loss: 0.1237
Validation Loss: 0.1189
Epoch: 40/130............. Train Loss: 0.1224
Validation Loss: 0.1170
Epoch: 50/130............. Train Loss: 0.1210
Validation Loss: 0.1159
Epoch: 60/130............. Train Loss: 0.1195
Validation Loss: 0.1146
Epoch: 70/130............. Train Loss: 0.1187
Validation Loss: 0.1147
Epoch: 80/130............. Train Loss: 0.1176
Validation Loss: 0.1145
Epoch: 90/130............. Train Loss: 0.1170
Validation Loss: 0.1152
Epoch: 100/130............. Train Loss: 0.1164
Validation Loss: 0.1172
Epoch: 110/130............. Train Loss: 0.1158
Validation Loss: 0.1190
Epoch: 120/130............. Train Loss: 0.1153
Validation Loss: 0.1213
Epoch: 130/130............. Train Loss: 0.1147
Validation Loss: 0.1224
Starting training of GRU model
Epoch: 10/130...........

C:\Users\sxiao15\AppData\Local\Temp\ipykernel_10904\4224545255.py:34: RuntimeWarning: invalid value encountered in longlong_scalars
  precision = TP/(TP + FP)


In [88]:
print(metrics_GRU)

                    A         B         C         D         E         F  \
accuracy     0.833333  0.666667  0.238636  0.231884  0.753846  0.708333   
sensitivity  0.166667  0.200000  0.042857  0.036364  0.000000  0.000000   
specificity  0.966667  0.869565  1.000000  1.000000  1.000000  1.000000   
precision    0.500000  0.400000  1.000000  1.000000       NaN       NaN   
AUC          0.822222  0.495652  0.719841  0.711688  0.300383  0.457516   

                    G         H         I         J  
accuracy     0.598485  0.572917  0.479695  0.819277  
sensitivity  0.306122  0.384615  0.641975  0.000000  
specificity  0.771084  0.701754  0.437700  1.000000  
precision    0.441176  0.468750  0.228070       NaN  
AUC          0.603639  0.541610  0.551690  0.548039  


In [18]:
# train_loss_LSTM_t2,val_loss_LSTM_t2,result_tabs_LSTM_t2,metrics_LSTM_t2 = project_cross_val(['shannon','bwpd','CST','Lactobacillus'], \
#                                                     train_ratio = 0.7,learn_rate = 0.01,hidden_dim = 15, n_layers = 2, device = device, \
#                                                     batch_size = 1000, drop_prob = 0,EPOCHS = 100, output_dim = 2, method = "LSTM", \
#                                                     out_var = "was_early_preterm",label_smooth= True, multi_outcome= True)

In [19]:
# print(metrics_LSTM_t2)